In [1]:
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [2]:
binance = Market.open("BN", "BTCBUSD")

In [3]:
binance.cache_duration

0

In [8]:
rbot.init_debug_log()
binance.reset_cache_duration()
print(binance.cache_duration)
binance.ohlcv(DAYS_BEFORE(10), DAYS_BEFORE(9), 60)

0
2022-12-17T04:56:45.572Z DEBUG [rbot::db::sqlite] update_cache_df 1670388960000000 -> 1670475405572730
2022-12-17T04:56:45.572Z DEBUG [rbot::db::sqlite] update cache duration 0
2022-12-17T04:56:45.666Z DEBUG [rbot::db::df] ohlc "2022-12-07T04:56:00.000000" -> "2022-12-08T04:56:45.572730"
2022-12-17T04:56:45.666Z DEBUG [rbot::db::df] Select from 2022-12-07T04:56:00.000000 -> 2022-12-08T04:56:45.572730


,open,high,low,close,volume
timestamp,,,,,
2022-12-07 04:56:00+00:00,17033.85,17038.45,17032.53,17035.01,83.09489
2022-12-07 04:57:00+00:00,17035.01,17039.44,17034.03,17038.60,54.92097
2022-12-07 04:58:00+00:00,17038.64,17044.36,17037.70,17043.04,90.13052
2022-12-07 04:59:00+00:00,17042.85,17044.94,17037.48,17041.07,65.74660
2022-12-07 05:00:00+00:00,17041.07,17042.73,17038.65,17039.87,75.39999
...,...,...,...,...,...
2022-12-08 04:52:00+00:00,16823.53,16826.29,16820.45,16821.64,64.44631
2022-12-08 04:53:00+00:00,16822.18,16822.25,16812.98,16816.60,83.12187
2022-12-08 04:54:00+00:00,16816.43,16818.14,16811.74,16816.22,96.99307


In [9]:
print(binance.cache_duration)
binance.ohlcv(DAYS_BEFORE(9), DAYS_BEFORE(8), 60)

86445572730
2022-12-17T04:57:27.610Z DEBUG [rbot::db::sqlite] update_cache_df 1670475420000000 -> 1670561847610044
2022-12-17T04:57:27.621Z DEBUG [rbot::db::df] ohlc "2022-12-08T04:57:00.000000" -> "2022-12-09T04:57:27.610044"
2022-12-17T04:57:27.621Z DEBUG [rbot::db::df] Select from 2022-12-08T04:57:00.000000 -> 2022-12-09T04:57:27.610044


,open,high,low,close,volume
timestamp,,,,,
2022-12-08 04:57:00+00:00,16830.12,16834.41,16828.02,16828.32,123.34969
2022-12-08 04:58:00+00:00,16828.30,16828.30,16825.41,16825.89,39.70606
2022-12-08 04:59:00+00:00,16826.33,16828.38,16812.05,16817.05,150.62231
2022-12-08 05:00:00+00:00,16817.05,16819.38,16815.71,16817.20,46.77757
2022-12-08 05:01:00+00:00,16817.07,16818.20,16812.30,16812.49,58.30779
...,...,...,...,...,...
2022-12-09 04:53:00+00:00,17193.10,17195.24,17191.81,17194.51,62.86832
2022-12-09 04:54:00+00:00,17194.03,17197.13,17193.03,17197.13,80.84095
2022-12-09 04:55:00+00:00,17197.10,17200.64,17195.88,17196.99,82.62162


In [10]:
print(binance.cache_duration)
binance.ohlcv(DAYS_BEFORE(8), DAYS_BEFORE(7), 60)

86445572730
2022-12-17T04:57:45.748Z DEBUG [rbot::db::sqlite] update_cache_df 1670561820000000 -> 1670648265748979
2022-12-17T04:57:45.748Z DEBUG [rbot::db::sqlite] update cache duration 86445572730
2022-12-17T04:57:45.764Z DEBUG [rbot::db::sqlite] Expire cache 2022-12-08T04:56:14.251021
2022-12-17T04:57:45.764Z DEBUG [rbot::db::df] Select from 2022-12-08T04:56:00.000000 -> 1970-01-01T00:00:00.000000
2022-12-17T04:57:46.055Z DEBUG [rbot::db::df] Select from 2022-12-08T04:56:00.000000 -> 1970-01-01T00:00:00.000000
2022-12-17T04:57:46.056Z DEBUG [rbot::db::df] ohlc "2022-12-09T04:57:00.000000" -> "2022-12-10T04:57:45.748979"
2022-12-17T04:57:46.056Z DEBUG [rbot::db::df] Select from 2022-12-09T04:57:00.000000 -> 2022-12-10T04:57:45.748979


,open,high,low,close,volume
timestamp,,,,,
2022-12-09 04:57:00+00:00,17197.74,17198.99,17195.69,17197.30,57.88931
2022-12-09 04:58:00+00:00,17197.51,17215.16,17197.47,17213.71,199.92491
2022-12-09 04:59:00+00:00,17213.36,17225.36,17213.03,17224.60,306.02495
2022-12-09 05:00:00+00:00,17224.90,17224.90,17219.96,17221.05,92.97029
2022-12-09 05:01:00+00:00,17221.02,17223.14,17217.72,17220.19,94.10492
...,...,...,...,...,...
2022-12-10 04:53:00+00:00,17155.57,17156.77,17154.65,17155.00,41.72162
2022-12-10 04:54:00+00:00,17154.88,17156.70,17153.86,17156.07,29.23255
2022-12-10 04:55:00+00:00,17156.15,17156.94,17153.87,17154.22,51.94949


In [11]:
binance.ohlcv(DAYS_BEFORE(10), DAYS_BEFORE(2), 60)


2022-12-17T04:58:08.960Z DEBUG [rbot::db::sqlite] update_cache_df 1670389080000000 -> 1671080288960697
2022-12-17T04:58:08.960Z DEBUG [rbot::db::sqlite] update cache duration 86445748979
2022-12-17T04:58:09.045Z DEBUG [rbot::db::sqlite] create iter 1 microsec
2022-12-17T04:58:10.353Z DEBUG [rbot::db::sqlite] load data before cache df1=(2395715, 4) df2=(19326026, 4)
2022-12-17T04:58:10.357Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-12-08T04:56:00.025000
2022-12-17T04:58:10.359Z DEBUG [rbot::db::df] merge len 2395715
2022-12-17T04:58:10.359Z DEBUG [rbot::db::df] merge df1=(2395715, 4)  df2=(19326026, 4)
2022-12-17T04:58:10.359Z DEBUG [rbot::db::sqlite] cache update diff before 2022-12-07T04:58:00.000000 -> 2022-12-08T04:56:00.000000
2022-12-17T04:58:10.359Z DEBUG [rbot::db::df] ohlcv_df, from=2022-12-07T04:58:00.000000 / to=2022-12-08T04:56:00.000000
2022-12-17T04:58:10.359Z DEBUG [rbot::db::df] Select from 2022-12-07T04:58:00.000000 -> 2022-12-08T04:56:00.00000

,open,high,low,close,volume
timestamp,,,,,
2022-12-07 04:58:00+00:00,17038.64,17044.36,17037.70,17043.04,90.13052
2022-12-07 04:59:00+00:00,17042.85,17044.94,17037.48,17041.07,65.74660
2022-12-07 05:00:00+00:00,17041.07,17042.73,17038.65,17039.87,75.39999
2022-12-07 05:01:00+00:00,17039.76,17042.96,17038.20,17040.20,56.53122
2022-12-07 05:02:00+00:00,17040.25,17047.10,17040.03,17046.81,65.18934
...,...,...,...,...,...
2022-12-15 04:54:00+00:00,17705.45,17708.95,17702.89,17707.28,41.93469
2022-12-15 04:55:00+00:00,17706.87,17708.97,17705.00,17706.33,26.73886
2022-12-15 04:56:00+00:00,17706.54,17706.60,17700.81,17702.33,26.50125


In [13]:
binance.cache_duration

0